In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
os.chdir('/content/drive/MyDrive/Throat cancer dataset')
print('we are inside the folder',os.getcwd() )

we are inside the folder /content/drive/MyDrive/Throat cancer dataset


In [3]:
data= '/content/drive/MyDrive/Throat cancer dataset'

In [4]:
train_data= data + '//train'
os.listdir(train_data)

['OSCC', 'Normal']

In [5]:
print('No. of images in OSCC:', len(os.listdir(train_data + '//OSCC')))
print('No. of images in Normal:', len(os.listdir(train_data + '//Normal')))

No. of images in OSCC: 2476
No. of images in Normal: 2370


In [6]:
test_path= data + "//test"
os.listdir(test_path)

['Normal', 'OSCC']

In [7]:
print('No. of images in OSCC:', len(os.listdir(test_path + '//OSCC')))
print('No. of images in Normal:', len(os.listdir(test_path + '//Normal')))

No. of images in OSCC: 95
No. of images in Normal: 31


In [8]:
import tensorflow
import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [9]:
IMAGE_SIZE= [224, 224]

In [10]:
model = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94781440/94765736 [==============================] - 0s 0us/step


In [11]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [12]:
# don't train existing weights
for layer in model.layers:
    layer.trainable = False
    

In [13]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Throat cancer dataset/train/*')

In [14]:
folders

['/content/drive/MyDrive/Throat cancer dataset/train/OSCC',
 '/content/drive/MyDrive/Throat cancer dataset/train/Normal']

In [15]:
# our layers - you can add more if you want
x = Flatten()(model.output)

In [16]:
prediction = Dense(1, activation='softmax', name= "FC")(x)

# create a model object
model = Model(inputs=model.input, outputs=prediction)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [18]:
#from tensorflow.keras.optimizers import Adam Adam(learning_rate=1e-4)

# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [19]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#In test data we don't do data augumentation

test_datagen = ImageDataGenerator(rescale = 1./255)

In [20]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_data,
                                                 target_size = IMAGE_SIZE,
                                                 batch_size = 128,
                                                 class_mode = 'categorical')   #class_mode = 'categorical' when we have more then
                                                                            # 2 classes

Found 4846 images belonging to 2 classes.


In [21]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = IMAGE_SIZE,
                                            batch_size = 128,
                                            class_mode = 'categorical')

Found 126 images belonging to 2 classes.


In [22]:
#Defining a custom callback function to stop training our model when accuracy goes above 99%
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau

class MyCallback(tensorflow.keras.callbacks.Callback):
  def on_epoch_end(self, epoc, logs={}):
    if logs.get('accuracy')>0.99:
      print("\n Reached accuracy...Stopped Traning")
      self.model.stop_training= True

my_callback= MyCallback()

#ReduceLROnPlateau to stabilize the training process of the model
rop_callback= ReduceLROnPlateau(moniter= "val_acc", patience=5, verbose=1, factor=0.2,min_lr=1e-4)

In [23]:
CALLBACKS= [rop_callback, my_callback]

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20
)

Epoch 1/20
33/38 [=========================>....] - ETA: 2:09 - loss: 1.2481 - accuracy: 0.5000